# XDMoD Data Analytics Framework — Machine Learning Example

Document version 3 (2025-07-18)

Compatible with XDMoD Data Analytics Framework v≥1.0.0 and <2.0.0

© 2023–2025 University at Buffalo Center for Computational Research

See the [xdmod-notebooks](https://github.com/ubccr/xdmod-notebooks) repository for information on setup, support, contributing, licensing, and referencing.

## Introduction
The XDMoD Data Analytics Framework provides API access to the data in XDMoD via the [xdmod_data](https://pypi.org/project/xdmod-data) Python package. This notebook provides an example showing how to use the `get_raw_data()` method to obtain and process raw job performance data, which is contained in the `SUPREMM` realm in XDMoD, and use the data to train a machine learning model to classify applications based on their performance characteristics.

The XDMoD Data Analytics Framework can be run either in an XDMoD-hosted JupyterHub (e.g., by clicking the "JupyterLab" button in ACCESS XDMoD) or locally on your machine.

## Install/upgrade the required modules
Run the code below to install/upgrade the packages needed to run this notebook.

In [ ]:
import sys
! {sys.executable} -m pip install --upgrade 'xdmod-data[report]>=1.0.0,<2.0.0' python-dotenv tabulate

If running that code caused a new version of Plotly to be installed/upgraded, you may need to refresh your browser window for plots to appear correctly.

## Configure notebook formatting

### Exceptions
Run the code below to simplify how Python exceptions are displayed in this notebook.

In [ ]:
import sys
def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)
get_ipython()._showtraceback = exception_handler

### Tables
Run the code below to set up for displaying Pandas DataFrames as Markdown tables in this notebook.

In [ ]:
from IPython.display import display, Markdown
def display_df_md_table(df):
    return display(Markdown(df.replace('\n', '<br/>', regex=True).to_markdown()))

### Plots
Run the code below to set up the external Plotly package to make plots using a custom XDMoD theme.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import xdmod_data.themes
pio.templates.default = 'timeseries'

## Prepare to authenticate with XDMoD

If you are running this notebook in an XDMoD-hosted JupyterHub (e.g., you clicked the "JupyterLab" button in ACCESS XDMoD), then authentication happens automatically and you can skip this section.

Otherwise, if you are running this notebook in a different Jupyter environment, you will need to obtain an API token from the XDMoD portal following [these instructions](https://github.com/ubccr/xdmod-data#api-token-access) and save it to a file that can be accessed by the Jupyter environment (e.g., in the home directory at `~/xdmod-data.env`) with the contents `XDMOD_API_TOKEN=token`, replacing `token` with your token. This file should be saved with `600` permissions (user read/write only). After you have done this, if you uncomment the last line of the code cell below and run it, it will read your token from `~/xdmod-data.env` into the environment, which will be used later when you start running methods from the API. It will print `True` if it successfully loaded the file.

In [ ]:
from dotenv import load_dotenv
from os.path import expanduser
from pathlib import Path
#load_dotenv(Path(expanduser('~/xdmod-data.env'), override=True))

## Initialize the DataWarehouse object

Run the code below to initialize a `DataWarehouse` object that will be used for making the API calls.

If you are running in an XDMoD-hosted JupyterHub, this object will make requests to the same XDMoD portal that is hosting the JupyterHub. To make requests to a different portal instead, you can specify the URL of that portal as a string parameter to the `DataWarehouse` constructor.

Otherwise, if you are running in a different Jupyter environment, you will need to specify the URL of the XDMoD portal as a string parameter to the `DataWarehouse` constructor (or set the `XDMOD_HOST` environment variable).

In [ ]:
from xdmod_data.warehouse import DataWarehouse
dw = DataWarehouse()

## Get the raw data

Run the code below to use the `get_raw_data()` method to request raw data from XDMoD and load them into a Pandas DataFrame. This example gets two months of job performance data for the ACCESS-allocated Bridges-2 resource. Each of the parameters of the `get_raw_data()` method will be explained later in this notebook. Use `with` to create a runtime context; this is also explained later in this notebook.

In [ ]:
with dw:
    df = dw.get_raw_data(
        duration=('2023-01-01', '2023-02-28'),
        realm='SUPREMM',
        fields=(
            'Application',
            'CPU User',
            'CPU User cov',
            'Wall Time',
            'Total memory used',
            'Net Ib0 Rx',
            'Net Ib0 Tx',
            'Memory Used Cov',
            'Net Ib0 Rx Cov',
            'Net Ib0 Tx Cov',
        ),
        filters={
            'Resource': [
                'Bridges 2 RM',
            ],
        },
        show_progress=True,
    )

## Inspect the data

In [ ]:
display(df)

## Prepare data for training

Get a list of the most used applications in the data set:

In [ ]:
application_counts = df['Application'].value_counts()
display(application_counts)

Looking at the list of applications, it should be noted:

* "NA" means the application name was not collected.
* "uncategorized" means the application name was collected but is not in the list of applications known to XDMoD.
* "PROPRIETARY" means the application is one whose software license does not allow performance data about the application to be reported in conjunction with the application's name.
* "python" and "r" could consist of multiple different codes running in the Python and R programming languages, rather than specific application packages.

With that in mind, only include jobs running the top eight individual application packages from the list:

In [ ]:
applications = ('orca', 'lammps', 'q-espresso', 'gromacs', 'specfem2d', 'namd', 'gdal', 'qmcchem')
df = df[df['Application'].isin(applications)]

In [ ]:
display(df)

One thing of which to be careful is that there may be some records that have `<NA>` values, indicating a particular record was not collected.

Filter out the jobs for which any value is `<NA>`:

In [ ]:
df = df.dropna()
display(df)

Convert the values from all the columns except "Application" from strings to floating point values:

In [ ]:
conversions = {}
for column in df.columns.drop('Application').values:
    conversions[column] = float
df = df.astype(conversions)

Use [scikit-learn](https://scikit-learn.org/) to split the jobs into a training set and test set:

In [ ]:
from sklearn.model_selection import train_test_split
X = df[[
    'CPU User',
    'CPU User cov',
    'Wall Time',
    'Total memory used',
    'Net Ib0 Rx',
    'Net Ib0 Tx',
    'Memory Used Cov',
    'Net Ib0 Rx Cov',
    'Net Ib0 Tx Cov',
]]
y = df['Application']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)
display(Markdown("Size of the test subset is **" + str(len(X_test)) + "**."))
display(Markdown("Size of the training subset is **" + str(len(X_train)) + "**."))

## Generate a Random Forest Classification model

Using the tools from `scikit-learn`, build a Random Forest Classification model.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(oob_score=True)
classifier.fit(X_train, y_train)
print('oob score:', classifier.oob_score_)

### Compute feature importances

In [ ]:
import pandas as pd
importances = pd.Series(
    classifier.feature_importances_,
    index=pd.Index(
        X_train.columns,
        name='Feature Name',
    ),
    name='Importance',
).sort_values(ascending=False)
display_df_md_table(importances)
plot = px.bar(importances, y='Importance')
plot.show()

### Create confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
matrix = confusion_matrix(y_test, classifier.predict(X_test), labels=applications)
cmd = ConfusionMatrixDisplay(matrix, display_labels=applications)
import matplotlib.pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
    'font.size': 16,
    'figure.figsize': (7, 7),
    'axes.labelpad': 20,
})
cmd.plot(
    colorbar=False,
    xticks_rotation='vertical',
    values_format='.0f',
    cmap='Blues',
)
# Export the figure as PDF
#plt.savefig('confusion_matrix.pdf', bbox_inches='tight')
plt.show()

## Details of the `get_raw_data()` method
Now that you have seen an example of using the `get_raw_data()` method, read below for more details on how it works.

### Wrap data warehouse calls in a runtime context
XDMoD data are accessed over a network connection, which involves establishing connections and creating temporary resources. To ensure these connections and resources are cleaned up properly in spite of any runtime errors, you should call data warehouse methods within a **runtime context** by using Python's `with` statement to wrap the execution of XDMoD queries, store the result, and execute any long running calculations outside of the runtime context, as in the template below.

In [ ]:
with dw:
    # XDMoD queries would go here
    pass
# Data processing would go here
pass

### Parameters
The `get_raw_data()` method has a number of parameters explained in detail below.

### Duration
The **duration** provides the time constraints of the data to be fetched from the XDMoD data warehouse.

As already seen, you can specify the duration as start and end dates. These are both inclusive, so if you only want one day of data, use the same date for start and end:

In [ ]:
with dw:
    data = dw.get_raw_data(
        duration=('2023-05-01', '2023-05-01'),
        realm='SUPREMM'
    )

You can instead specify the duration using a special string value; a list of the valid values can be obtained by calling the `get_durations()` method.

In [ ]:
with dw:
    durations = dw.get_durations()
display(durations)

### Realm
A **realm** is a category of data in the XDMoD data warehouse. You can use the `describe_raw_realms()` method to get a DataFrame containing the list of realms for which raw data are available.

In [ ]:
with dw:
    raw_realms = dw.describe_raw_realms()
display_df_md_table(raw_realms)

### Fields
A **field** is a measurement for which raw data exist in a given realm. You can use the `describe_raw_fields(realm)` method to get a DataFrame containing the list of valid fields in the given realm. The realm must be passed in as a string.

In [ ]:
with dw:
    raw_fields = dw.describe_raw_fields('SUPREMM')
display_df_md_table(raw_fields)

### Filters
**Filters** allow you to include only data that have certain values for given **dimensions**, which are groupings of data. You can use the `describe_dimensions(realm)` method to get a DataFrame containing the list of valid dimensions in the given realm. The realm must be passed in as a string and can be either the ID or the label of the realm.

In [ ]:
with dw:
    dimensions = dw.describe_dimensions('SUPREMM')
display_df_md_table(dimensions)

You can use the `get_filter_values(realm, dimension)` method to get a DataFrame containing the list of valid filter values for the given dimension in the given realm. The realm and dimension must be passed in as strings.

In [ ]:
with dw:
    filter_values = dw.get_filter_values('SUPREMM', 'Resource') # 'resource' also works
display_df_md_table(filter_values)

For methods in the API that take filters as arguments, you must specify the filters as a dictionary in which the keys are dimensions (labels or IDs) and the values are string filter values (labels or IDs) or sequences of string filter values. For example, to return only data for which the field of science is Materials Engineering and the resource is either Bridges-2 RM or TACC Stampede2:

In [ ]:
with dw:
    df = dw.get_raw_data(
        duration=('2021-05-01', '2021-05-01'),
        realm='SUPREMM',
        filters={
            'Field of Science': 'Materials Engineering', # 'fieldofscience': '177' also works
            'Resource': ( # 'resource' also works
                'Bridges 2 RM', # '2900' also works
                'STAMPEDE2 TACC', # '2825' also works
            ),
        },
        show_progress=True
    )

### Show progress
Set the `show_progress` parameter to `True` to periodically print how many rows have been gotten so far. 

## Additional Examples
For additional examples, please see the [xdmod-notebooks repository](https://github.com/ubccr/xdmod-notebooks).

In [ ]:
# This cell is used to create the footer of this notebook.
from xdmod_data.report import footer
footer({
    'history': [
        ['1', '2023-07-21', 'Initial version'],
        ['2', '2024-09-27', 'Replace regression example with classification example'],
        ['3', '2025-07-18', 'Update for JupyterHub support'],
    ],
})